In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime
import dask.dataframe as dd

In [4]:
os.chdir('E:\personal project\Lending Club')

In [ ]:
#dff = dd.read_csv('LoanStats_2015.csv',dtype='str')
#dff.loan_status.value_counts().compute()
#df=dff[dff.loan_status.isin(['Fully Paid','Charged Off','Late (31-120 days),','Default','In Grace Period','Late (16-30 days)'])].compute()
#df.reset_index(inplace=True,drop=True)

In [ ]:
#df.to_csv('LoanStats_2015_nocurrent.csv',index=False)

In [19]:
df = pd.read_csv('LoanStats_2015_nocurrent.csv',dtype='str',encoding='latin-1')

In [ ]:
df.shape

In [ ]:
delete_col=df.isnull().sum()[df.isnull().sum()/len(df)>0.5].index.tolist()

In [ ]:
df.drop(delete_col,inplace=True,axis=1)

In [ ]:
#date_iss=df.issue_d

In [ ]:
#date_iss=datetime.strptime(df.issue_d[0], '%b-%y')
#date_ear=datetime.strptime(df.earliest_cr_line[0], '%b-%y')
#date_la_pull=datetime.strptime(df.last_credit_pull_d[0], '%b-%y')

In [ ]:
#df.last_credit_pull_d

In [ ]:
#date_iss.year-date_ear.year

In [ ]:
#df.issue_d[0]-df.earliest_cr_line[0]

In [ ]:
# variables will not be used for applications model
not_used_cols=['funded_amnt',
'funded_amnt_inv',
'issue_d',
'pymnt_plan',
'out_prncp',
'out_prncp_inv',
'total_pymnt',
'total_pymnt_inv',
'total_rec_prncp',
'total_rec_int',
'total_rec_late_fee',
'recoveries',
'collection_recovery_fee',
'last_pymnt_d',
'last_pymnt_amnt',
'collections_12_mths_ex_med',
'policy_code',
'delinq_amnt',
'hardship_flag',
'debt_settlement_flag']

In [ ]:
df.drop(not_used_cols,inplace=True,axis=1)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(25,5)) 
sns.set_context(font_scale=10)   
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis',xticklabels=list(range(len(df.columns))))

In [ ]:
df.isnull().sum()[df.isnull().sum()>0]

In [23]:
# remove units
df.int_rate=df.int_rate.str[:-1]
df.revol_util=df.revol_util.str[:-1]
df.term=df.term.str[:3]

df.emp_length[(df.emp_length!='1 year')&(df.emp_length!='< 1 year')&(df.emp_length!='10+ years')]=df.emp_length[(df.emp_length!='1 year')&(df.emp_length!='< 1 year')&(df.emp_length!='10+ years')].str[:-6]
df.emp_length[df.emp_length=='< 1 year']=df.emp_length[df.emp_length=='< 1 year'].str[:-5]
df.emp_length[df.emp_length=='1 year']=df.emp_length[df.emp_length=='1 year'].str[:-5]
df.emp_length[df.emp_length=='10+ years']=df.emp_length[df.emp_length=='10+ years'].str[:-7]
# '<1 year'-->0, '10+ years'-->10
df.emp_length[df.emp_length=='< 1']=0
df.emp_length[df.emp_length=='']=np.nan

# keep first 3 digit of zipcode
df.zip_code=df.zip_code.str[:-2]
# bin loan titles with small count into Other
#df.title[df.title.isin(df.title.value_counts()[df.title.value_counts()<100].index.tolist())==True]='Other'

# transform grade and sub_grade into ordinal numbers
#grade=df.grade.value_counts().index.sort_values().tolist()
#for i in range(len(grade)):
#    df.grade[df.grade==grade[i]]=i

#s_grade=df.sub_grade.value_counts().index.sort_values().tolist()
#for i in range(len(s_grade)):
#    df.sub_grade[df.sub_grade==s_grade[i]]=np.floor(i/5)+np.mod(i,5)/10*2
# Current,Fully Paid-->not default(0), -->default(1)

In [ ]:
# good loans(0) and bad loan(1) 
df.loan_status.value_counts()

In [ ]:
df.emp_length.value_counts()

In [ ]:
## need to be transformed
'earliest_cr_line','last_credit_pull_d'

In [24]:
l_int=['loan_amnt','term','emp_length','annual_inc','zip_code','delinq_2yrs','inq_last_6mths',
      'mths_since_last_delinq','open_acc','pub_rec','revol_bal','total_acc','acc_now_delinq','tot_coll_amt',
      'tot_cur_bal','total_rev_hi_lim','acc_open_past_24mths','avg_cur_bal', 'bc_open_to_buy','chargeoff_within_12_mths', 
       'mo_sin_old_il_acct','mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl','mort_acc', 
       'mths_since_recent_bc', 'mths_since_recent_inq','num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl','num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 
       'num_tl_120dpd_2m','num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m','pub_rec_bankruptcies','tax_liens', 
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit','total_il_high_credit_limit']
l_float=['int_rate','installment','dti','revol_util','bc_util','pct_tl_nvr_dlq', 'percent_bc_gt_75']

In [25]:
for i in l_int:
    print(i)
    df[i]=df[i].astype(int)
for i in l_float:
    df[i]=df[i].astype(float)

loan_amnt
term
emp_length


ValueError: cannot convert float NaN to integer

In [ ]:
cat_var=df.select_dtypes(include='')
num_var=

In [ ]:
dff.head()